<a href="https://colab.research.google.com/github/RogerHeederer/DL_Entry_Pytorch/blob/master/CostFuct_CrossEntropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ref: https://wikidocs.net/60572

소프트맥스 회귀의 비용 함수 구현

In [3]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

소프트맥스 비용 함수 로우-레벨로 구현

In [4]:
z = torch.FloatTensor([1, 2, 3]) #3개의 원소를 가진 벡터 텐서 정의
hypothesis = F.softmax(z, dim=0)
print(hypothesis)
# 3개의 원소 값들이 0~1사이의 값들로 변환됨

tensor([0.0900, 0.2447, 0.6652])


In [5]:
hypothesis.sum()

tensor(1.)

In [6]:
# 비용 함수 구해보기
z = torch.rand(3, 5, requires_grad=True) # 3개의 샘플/ 그리고 5개의 클래스
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


위 결과는, 샘플은 3개. 각각의 샘플이 5개의 클래스 중 어디에 가장 높은 확률로 속하는 것인지 예시로 나타낸 것

In [7]:
# 각 샘플에 대해 임의의 레이블 만들기(정수인코딩?)
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [8]:
y.unsqueeze(1)

tensor([[0],
        [2],
        [1]])

In [10]:
#원핫 인코딩 수행. 3X5 형태로 0을 쭉 채움
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [11]:
#첫번째 인자 dim=1, y.unsqueeze(1)에서 나타내는 숫자를 위치로 삼아서, 세번째 인자값(1)을 넣어준다
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # scatter_ 언더 라인을 붙이면 덮어쓰기 연산이다.
y_one_hot

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [12]:
#비용함수
cost = ( y_one_hot * -torch.log(hypothesis) ).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


파이토치로 소프트맥스의 비용 함수 구현( 하이-레벨)

In [14]:
# low level
# torch.log(F.softmax(z, dim=1))
-F.log_softmax(z, dim=1) # high level

tensor([[1.3301, 1.8084, 1.6846, 1.3530, 2.0584],
        [1.4147, 1.8174, 1.4602, 1.6450, 1.7758],
        [1.5025, 1.6165, 1.4586, 1.8360, 1.6776]], grad_fn=<NegBackward>)

In [15]:
y_one_hot

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [16]:
(y_one_hot * -F.log_softmax(z, dim=1))

tensor([[1.3301, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.4602, 0.0000, 0.0000],
        [0.0000, 1.6165, 0.0000, 0.0000, 0.0000]], grad_fn=<MulBackward0>)

In [17]:
(y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

F.cross_entropy는 비용 함수에 소프트맥수 함수까지 포함하고 있다

In [18]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)